In [23]:
import pandas as pd
import numpy as np

In [24]:
df=pd.read_csv('../data/processed_train_filled.csv')

In [25]:
edu_df = df['education'].map({'illiterate':0,'basic.4y':1,'basic.6y':2,'basic.9y':3,
                                  'high.school':4,'professional.course':5,'university.degree':6})
job_df = df['job'].map({'unemployed': 0,'student': 0,'retired': 0,'housemaid': 0,'entrepreneur': 1,
                              'self-employed': 1,'blue-collar': 2,'services': 3,'technician': 4,'admin.': 5,'management': 5})
def_df = df['default'].map({'yes':1,'no':0})
housing_df = df['housing'].map({'yes':1,'no':0})
loan_df = df['loan'].map({'yes':1,'no':0})
debt_df = pd.DataFrame(columns=['debt'])
debt_df['debt'] = def_df + housing_df + loan_df
train_df = pd.concat([edu_df, job_df, debt_df], axis=1)

In [26]:
mon_df = df['month'].map({'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12})
day_df = df['day_of_week'].map({'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5})
train_df = pd.concat([train_df, mon_df, day_df], axis=1)

In [27]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
numeric_X = df[['age', 'campaign', 'pdays', 'previous', 'duration']]
train_df = pd.concat([train_df, pd.DataFrame(scaler.fit_transform(numeric_X), columns = numeric_X.columns)], axis=1)
train_df

,education,job,debt,month,day_of_week,age,campaign,pdays,previous,duration
0,5,5,2,8,1,0.876440,-0.327368,-2.024892,0.356261,2.425786
1,4,3,1,5,1,0.793699,-0.327368,-1.107256,0.356261,2.491410
2,3,2,0,4,3,0.628215,-0.465802,0.773898,-0.164927,-0.680888
3,4,1,3,8,5,-1.192102,3.133474,0.685193,-0.686116,-0.549639
4,6,5,0,11,2,0.379990,-0.327368,-1.633367,1.398639,1.418386
...,...,...,...,...,...,...,...,...,...,...
22495,4,5,0,8,5,-0.612910,-0.050501,0.461901,0.877450,2.013890
22496,4,5,1,8,5,-0.530168,1.472270,-0.737143,0.877450,1.423273
22497,5,5,1,5,4,-1.274843,-0.465802,0.055083,1.919828,0.849411
22498,4,0,1,5,2,1.372890,0.226367,0.657664,-0.686116,-0.398146


In [28]:
y = df['subscribe'].map({'yes':1,'no':0})

In [29]:
# import train_test split
from sklearn.model_selection import train_test_split
# split data into train and test, 80% in training and 20% in testing
X_train, X_test, y_train, y_test = train_test_split(train_df, y, test_size=0.2,
                                                    random_state = 99)

In [30]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

X_with_intercept = sm.add_constant(X_train)
stats_model = sm.OLS(y_train, X_with_intercept)
results = stats_model.fit()

print("P-values:")
print(results.pvalues)


P-values:
const          1.374752e-05
education      7.398865e-15
job            4.736404e-10
debt           2.535996e-04
month          4.585310e-07
day_of_week    1.024654e-01
age            1.603796e-12
campaign       4.565386e-74
pdays          9.350059e-35
previous       1.233935e-02
duration       2.032486e-01
dtype: float64


In [31]:
coeff_df = pd.DataFrame(lin_reg.coef_, train_df.columns, columns=['Coefficient'])
coeff_df

,Coefficient
education,0.012509
job,-0.009499
debt,0.013943
month,0.005996
day_of_week,0.002861
age,0.018316
campaign,0.047631
pdays,-0.039107
previous,-0.007994
duration,-0.003957
